In [25]:
import numpy as np
import os
import time

def load_all_scp_files(folder_path):
    """
    Charge tous les fichiers SCP d'un dossier et retourne un dictionnaire
    où les clés sont les noms des fichiers et les valeurs sont les listes de sous-ensembles.
    """
    all_subsets = {}

    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".txt"):  # S'assurer de ne traiter que les fichiers texte
            file_path = os.path.join(folder_path, filename)

            with open(file_path, "r") as file:
                lines = file.readlines()

            # Lire la première ligne : nombre d'éléments et nombre de sous-ensembles
            num_elements, num_subsets = map(int, lines[0].split())

            # Lire les sous-ensembles
            subsets = [set(map(int, line.strip().split())) for line in lines[1:num_subsets+1]]

            # Stocker les sous-ensembles dans le dictionnaire
            all_subsets[filename] = subsets

    return all_subsets



def fitness_func(solution, subsets, universe_size):
    """Évalue une solution en mesurant le nombre d'éléments couverts"""
    covered_elements = set()
    
    for idx, selected in enumerate(solution):
        if selected:
            covered_elements.update(subsets[idx])
    return len(covered_elements), len(covered_elements) / universe_size  # Nombre d'éléments couverts, score normalisé

def update_velocity_binary(velocity, particle, personal_best, global_best, w, c1, c2):
    """Mise à jour de la vitesse pour le PSO binaire"""
    r1, r2 = np.random.rand(len(particle)), np.random.rand(len(particle))
    new_velocity = (w * velocity +
                    c1 * r1 * (personal_best - particle) +
                    c2 * r2 * (global_best - particle))
    return np.clip(new_velocity, -4, 4)  # Valeur de saturation adaptée au PSO binaire

def binary_update(particles, velocities):
    """Mise à jour des particules en utilisant la transformation sigmoïde"""
    prob = 1 / (1 + np.exp(-velocities))
    return (np.random.rand(*particles.shape) < prob).astype(int)

def pso_mcp(subsets, universe_size, num_subsets, num_particles=100, num_iterations=50, 
            c1=2, c2=2, w=1, vmax=10, early_stop=30):
    """Optimisation du Maximum Covering Problem avec PSO binaire"""
    np.random.seed(42)
    particles = np.random.choice([0, 1], size=(num_particles, num_subsets))
    velocities = np.zeros((num_particles, num_subsets))
    personal_best = particles.copy()
    personal_best_scores = [fitness_func(p, subsets, universe_size) for p in particles]
    global_best = max(personal_best, key=lambda p: fitness_func(p, subsets, universe_size)[1])
    global_best_covered, global_best_score = fitness_func(global_best, subsets, universe_size)
    
    best_scores = []
    no_improvement = 0  # Compteur pour l'arrêt anticipé
    start_time = time.time()
    
    for iteration in range(num_iterations):
        for i in range(num_particles):
            velocities[i] = update_velocity_binary(velocities[i], particles[i], personal_best[i], global_best, w, c1, c2)
            particles[i] = binary_update(particles[i], velocities[i])
            
            num_covered, score = fitness_func(particles[i], subsets, universe_size)
            
            if score > personal_best_scores[i][1]:
                personal_best[i] = particles[i].copy()
                personal_best_scores[i] = (num_covered, score)
            
            if score > global_best_score:
                global_best = particles[i].copy()
                global_best_covered, global_best_score = num_covered, score
                no_improvement = 0  # Réinitialiser l'arrêt anticipé
            
        best_scores.append(global_best_score)
        print(f"Iteration {iteration+1}/{num_iterations} - Covered: {global_best_covered}, Score: {global_best_score:.4f}")
        
        if no_improvement >= early_stop:
            print(f"⏹️ Arrêt anticipé après {iteration+1} itérations.")
            break
        
        no_improvement += 1
    
    execution_time = time.time() - start_time
    print(f"✅ Optimisation terminée en {execution_time:.2f} sec - Meilleure couverture: {global_best_covered}, Score final: {global_best_score:.4f}")
    return global_best, global_best_covered, global_best_score, execution_time

def run_experiments():
    """Exécute l'optimisation PSO sur tous les benchmarks"""
    benchmark_folder = "Benchmark"
    subsets_data = load_all_scp_files(benchmark_folder)
    results = []
    
    for file, subsets in subsets_data.items():
        universe_size = max(max(subset) for subset in subsets)
        num_subsets = len(subsets)

        print(f"\n🚀 Exécution de PSO sur {file}...")
        best_solution, best_covered, best_score, exec_time = pso_mcp(subsets, universe_size, num_subsets)
        results.append((file, best_solution, best_covered, best_score, exec_time))
    
    print("\n📊 Résumé des résultats :")
    for file, _, best_cov, score, time in results:
        print(f"{file}: Couvert = {best_cov}, Score = {score:.4f}, Temps = {time:.2f}s")

if __name__ == "__main__":
    run_experiments()



🚀 Exécution de PSO sur scp41.txt...
Iteration 1/50 - Covered: 934, Score: 0.9340
Iteration 2/50 - Covered: 941, Score: 0.9410
Iteration 3/50 - Covered: 945, Score: 0.9450
Iteration 4/50 - Covered: 957, Score: 0.9570
Iteration 5/50 - Covered: 957, Score: 0.9570
Iteration 6/50 - Covered: 962, Score: 0.9620
Iteration 7/50 - Covered: 964, Score: 0.9640
Iteration 8/50 - Covered: 974, Score: 0.9740
Iteration 9/50 - Covered: 974, Score: 0.9740
Iteration 10/50 - Covered: 974, Score: 0.9740
Iteration 11/50 - Covered: 974, Score: 0.9740
Iteration 12/50 - Covered: 974, Score: 0.9740
Iteration 13/50 - Covered: 974, Score: 0.9740
Iteration 14/50 - Covered: 976, Score: 0.9760
Iteration 15/50 - Covered: 979, Score: 0.9790
Iteration 16/50 - Covered: 979, Score: 0.9790
Iteration 17/50 - Covered: 983, Score: 0.9830
Iteration 18/50 - Covered: 983, Score: 0.9830
Iteration 19/50 - Covered: 984, Score: 0.9840
Iteration 20/50 - Covered: 984, Score: 0.9840
Iteration 21/50 - Covered: 987, Score: 0.9870
Iterat

KeyboardInterrupt: 